Import Libraries and start spark session.\
\
then, see data example, data types and some unwanted strings 

In [ ]:
from pyspark.sql import (SparkSession, 
						types as t, 
            			functions as F, 
            			Window,
						)
import pyspark.pandas as ps

import numpy as np

spark = SparkSession.builder.appName('etl_with_pyspark').getOrCreate()

In [34]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahmedmohamed2003/cafe-sales-dirty-data-for-cleaning-training")

# print("Path to dataset files:", path)

filepath = path+'/dirty_cafe_sales.csv'

df = spark.read.csv(filepath,header=True)

df.head(5), df.dtypes, df.select('Item').drop_duplicates().toPandas()

([Row(Transaction ID='TXN_1961373', Item='Coffee', Quantity='2', Price Per Unit='2.0', Total Spent='4.0', Payment Method='Credit Card', Location='Takeaway', Transaction Date='2023-09-08'),
  Row(Transaction ID='TXN_4977031', Item='Cake', Quantity='4', Price Per Unit='3.0', Total Spent='12.0', Payment Method='Cash', Location='In-store', Transaction Date='2023-05-16'),
  Row(Transaction ID='TXN_4271903', Item='Cookie', Quantity='4', Price Per Unit='1.0', Total Spent='ERROR', Payment Method='Credit Card', Location='In-store', Transaction Date='2023-07-19'),
  Row(Transaction ID='TXN_7034554', Item='Salad', Quantity='2', Price Per Unit='5.0', Total Spent='10.0', Payment Method='UNKNOWN', Location='UNKNOWN', Transaction Date='2023-04-27'),
  Row(Transaction ID='TXN_3160411', Item='Coffee', Quantity='2', Price Per Unit='2.0', Total Spent='4.0', Payment Method='Digital Wallet', Location='In-store', Transaction Date='2023-06-11')],
 [('Transaction ID', 'string'),
  ('Item', 'string'),
  ('Quan

All columns is string, replace 'ERROR','UNKNOWN','None' with None first

In [3]:
df = df.replace(['ERROR', 'UNKNOWN','None'], [None,None,None])

Rename column for more suitable name, and also change to appropiate type

In [4]:
df = df.toDF('transaction_id',
               	'item_name',
         		'item_quantity',
     			'item_ppu',
				'item_total_price',
				'payment_type',
				'location_type',
				'transaction_date'
			   )

df = df.withColumn("item_quantity", F.col("item_quantity").cast("integer")) \
       	.withColumn("item_ppu", F.col("item_ppu").cast("double")) \
       	.withColumn("item_total_price", F.col("item_total_price").cast("double"))\
		.withColumn("transaction_date", F.col("transaction_date").cast("date"))

df.show()

+--------------+---------+-------------+--------+----------------+--------------+-------------+----------------+
|transaction_id|item_name|item_quantity|item_ppu|item_total_price|  payment_type|location_type|transaction_date|
+--------------+---------+-------------+--------+----------------+--------------+-------------+----------------+
|   TXN_1961373|   Coffee|            2|     2.0|             4.0|   Credit Card|     Takeaway|      2023-09-08|
|   TXN_4977031|     Cake|            4|     3.0|            12.0|          Cash|     In-store|      2023-05-16|
|   TXN_4271903|   Cookie|            4|     1.0|            NULL|   Credit Card|     In-store|      2023-07-19|
|   TXN_7034554|    Salad|            2|     5.0|            10.0|          NULL|         NULL|      2023-04-27|
|   TXN_3160411|   Coffee|            2|     2.0|             4.0|Digital Wallet|     In-store|      2023-06-11|
|   TXN_2602893| Smoothie|            5|     4.0|            20.0|   Credit Card|         NULL| 

In [5]:
df.printSchema()


root
 |-- transaction_id: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- item_quantity: integer (nullable = true)
 |-- item_ppu: double (nullable = true)
 |-- item_total_price: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- location_type: string (nullable = true)
 |-- transaction_date: date (nullable = true)



+ No Null value in transaction_id


In [6]:
df.filter(F.col('transaction_id').isNull()).count()

0

See if we can fill Null with something\
Note: payment_type, location_type, transaction_date can't be filled since we can't resolve it\

1. insert ppu for know item name first
2. calculate numeric fields
3. resolve item_name from item_ppu if it is unique and not null


In [7]:
df.select('item_name','item_ppu').dropna().dropDuplicates().orderBy('item_ppu').show()

+---------+--------+
|item_name|item_ppu|
+---------+--------+
|   Cookie|     1.0|
|      Tea|     1.5|
|   Coffee|     2.0|
|    Juice|     3.0|
|     Cake|     3.0|
| Sandwich|     4.0|
| Smoothie|     4.0|
|    Salad|     5.0|
+---------+--------+



In [8]:
# check with 16 - 3 cases from desktop

# df = spark.read.csv("book1.csv",header=True)
# df = df.withColumn("item_quantity", F.col("item_quantity").cast("integer")) \
#        	.withColumn("item_ppu", F.col("item_ppu").cast("double")) \
#        	.withColumn("item_total_price", F.col("item_total_price").cast("double"))


In [9]:
# item_name Null with item_ppu in [1.0, 1.5, 2.0, 5.0] can be replaced with the actual item_name. 
df = df.withColumn("item_ppu", 
                   	F.when(F.col("item_ppu").isNull(),
						F.when(F.col("item_name") == 'Cookie', 1)
						.when(F.col("item_name") == 'Tea', 1.5)
						.when(F.col("item_name") == 'Coffee', 2)
						.when(F.col("item_name") == 'Juice', 3)
						.when(F.col("item_name") == 'Cake', 3)
						.when(F.col("item_name") == 'Sandwich', 4)
						.when(F.col("item_name") == 'Smoothie', 4)
           				.when(F.col("item_name") == 'Salad', 5)
            			)
					.otherwise(F.col("item_ppu"))
					)

In [10]:
df = df.withColumn('item_total_price',
                    F.when(F.col("item_total_price").isNull(),df.item_quantity * df.item_ppu)
						.otherwise(df.item_total_price)
            		)
df = df.withColumn('item_ppu',
                    F.when(F.col("item_ppu").isNull(),df.item_total_price / df.item_quantity)
						.otherwise(df.item_ppu)
            		)
df = df.withColumn('item_quantity',
                    F.when(F.col("item_quantity").isNull(),df.item_total_price / df.item_ppu)
						.otherwise(df.item_quantity)
            		)



In [11]:
# item_name Null with item_ppu in [1.0, 1.5, 2.0, 5.0] can be replaced with the actual item_name. 
df = df.withColumn("item_name", 
                   	F.when(F.col("item_name").isNull(),
						F.when(F.col("item_ppu") == 1.0, 'Cookie')
						.when(F.col("item_ppu") == 1.5, 'Tea')
						.when(F.col("item_ppu") == 2.0, 'Coffee')
						.when(F.col("item_ppu") == 5.0, 'Salad')
            			)
					.otherwise(F.col("item_name"))
					)

df.select('item_name','item_ppu').dropna().dropDuplicates(['item_ppu']).orderBy('item_ppu').show()

+ Recalculate item_total_price, item_ppu, item_quantity
	+ from item_total_price = item_quantity * item_ppu

In [12]:
df.show()

+--------------+---------+-------------+--------+----------------+--------------+-------------+----------------+
|transaction_id|item_name|item_quantity|item_ppu|item_total_price|  payment_type|location_type|transaction_date|
+--------------+---------+-------------+--------+----------------+--------------+-------------+----------------+
|   TXN_1961373|   Coffee|          2.0|     2.0|             4.0|   Credit Card|     Takeaway|      2023-09-08|
|   TXN_4977031|     Cake|          4.0|     3.0|            12.0|          Cash|     In-store|      2023-05-16|
|   TXN_4271903|   Cookie|          4.0|     1.0|             4.0|   Credit Card|     In-store|      2023-07-19|
|   TXN_7034554|    Salad|          2.0|     5.0|            10.0|          NULL|         NULL|      2023-04-27|
|   TXN_3160411|   Coffee|          2.0|     2.0|             4.0|Digital Wallet|     In-store|      2023-06-11|
|   TXN_2602893| Smoothie|          5.0|     4.0|            20.0|   Credit Card|         NULL| 

In [13]:
# checl if some case still be improved

# df.select('item_name').dropDuplicates().show()
# df.filter(df.item_name == np.NaN).show()
# df.filter(F.col("item_name").isNull()).orderBy('item_total_price').show()
# df.orderBy('item_total_price').show()

(df
# .filter(~F.col("item_name").isNull())
# .filter(F.col("item_quantity").isNull())
# .filter(F.col("item_ppu").isNull())
# .filter(F.col("item_total_price").isNull())
# .filter(F.col("item_ppu") == 5)
# .orderBy('item_ppu').show()

).show()

+--------------+---------+-------------+--------+----------------+--------------+-------------+----------------+
|transaction_id|item_name|item_quantity|item_ppu|item_total_price|  payment_type|location_type|transaction_date|
+--------------+---------+-------------+--------+----------------+--------------+-------------+----------------+
|   TXN_1961373|   Coffee|          2.0|     2.0|             4.0|   Credit Card|     Takeaway|      2023-09-08|
|   TXN_4977031|     Cake|          4.0|     3.0|            12.0|          Cash|     In-store|      2023-05-16|
|   TXN_4271903|   Cookie|          4.0|     1.0|             4.0|   Credit Card|     In-store|      2023-07-19|
|   TXN_7034554|    Salad|          2.0|     5.0|            10.0|          NULL|         NULL|      2023-04-27|
|   TXN_3160411|   Coffee|          2.0|     2.0|             4.0|Digital Wallet|     In-store|      2023-06-11|
|   TXN_2602893| Smoothie|          5.0|     4.0|            20.0|   Credit Card|         NULL| 

In the fact table column Item, Payment Method, Location will be replace with id\
+ let's start with creating dim_item




EXTRA 1.select only necessary columns for dim tables +
	

In [14]:
# EXTRA: 	select only necessary columns for dim tables
#			+ drop Null values
#			+ cache
# to improve performance on creating dimension tables 
# (multiple use of distinct = shuffle data between partitions = high cost?)

dim_all = (df.select('item_name','item_ppu','payment_type','location_type')
           	.dropna()
            .distinct()
			)
dim_all.cache()

DataFrame[item_name: string, item_ppu: double, payment_type: string, location_type: string]

In [15]:
dim_item = dim_all.select('item_name','item_ppu').distinct()
w = Window.orderBy(dim_item.item_name)
dim_item = dim_item.withColumn('item_id', F.row_number().over(w)).select('item_id','item_name','item_ppu')
dim_item.show()

+-------+---------+--------+
|item_id|item_name|item_ppu|
+-------+---------+--------+
|      1|     Cake|     3.0|
|      2|   Coffee|     2.0|
|      3|   Cookie|     1.0|
|      4|    Juice|     3.0|
|      5|    Salad|     5.0|
|      6| Sandwich|     4.0|
|      7| Smoothie|     4.0|
|      8|      Tea|     1.5|
+-------+---------+--------+



+ create dim_payment

In [16]:
dim_payment = dim_all.select('payment_type').distinct()
w = Window.orderBy(dim_payment.payment_type)
dim_payment = dim_payment.withColumn('payment_type_id', F.row_number().over(w)).select('payment_type_id','payment_type')
dim_payment.show()

+---------------+--------------+
|payment_type_id|  payment_type|
+---------------+--------------+
|              1|          Cash|
|              2|   Credit Card|
|              3|Digital Wallet|
+---------------+--------------+



+ create dim_location

In [17]:
dim_location = dim_all.select('location_type').distinct()
w = Window.orderBy(dim_location.location_type)
dim_location = dim_location.withColumn('location_type_id', F.row_number().over(w)).select('location_type_id','location_type')
dim_location.show()

dim_all.unpersist()

+----------------+-------------+
|location_type_id|location_type|
+----------------+-------------+
|               1|     In-store|
|               2|     Takeaway|
+----------------+-------------+



DataFrame[item_name: string, item_ppu: double, payment_type: string, location_type: string]

Join Dimension table to df

In [18]:
# Use F.broadcast on small table in joining
df = df.join(F.broadcast(dim_item), on="item_name", how="left")
df = df.join(F.broadcast(dim_payment), on="payment_type", how="left")
df = df.join(F.broadcast(dim_location), on="location_type", how="left")
df.show()


+-------------+--------------+---------+--------------+-------------+--------+----------------+----------------+-------+--------+---------------+----------------+
|location_type|  payment_type|item_name|transaction_id|item_quantity|item_ppu|item_total_price|transaction_date|item_id|item_ppu|payment_type_id|location_type_id|
+-------------+--------------+---------+--------------+-------------+--------+----------------+----------------+-------+--------+---------------+----------------+
|     Takeaway|   Credit Card|   Coffee|   TXN_1961373|          2.0|     2.0|             4.0|      2023-09-08|      2|     2.0|              2|               2|
|     In-store|          Cash|     Cake|   TXN_4977031|          4.0|     3.0|            12.0|      2023-05-16|      1|     3.0|              1|               1|
|     In-store|   Credit Card|   Cookie|   TXN_4271903|          4.0|     1.0|             4.0|      2023-07-19|      3|     1.0|              2|               1|
|         NULL|       

In [19]:
# rearrange columns
fact_order = df.select(
    "transaction_id",
    "item_id",
    # "item_name",
    "item_quantity",
    # "item_ppu",
    "item_total_price",
    "payment_type_id",
    "location_type_id",
    "transaction_date"
)
fact_order.show()


+--------------+-------+-------------+----------------+---------------+----------------+----------------+
|transaction_id|item_id|item_quantity|item_total_price|payment_type_id|location_type_id|transaction_date|
+--------------+-------+-------------+----------------+---------------+----------------+----------------+
|   TXN_1961373|      2|          2.0|             4.0|              2|               2|      2023-09-08|
|   TXN_4977031|      1|          4.0|            12.0|              1|               1|      2023-05-16|
|   TXN_4271903|      3|          4.0|             4.0|              2|               1|      2023-07-19|
|   TXN_7034554|      5|          2.0|            10.0|           NULL|            NULL|      2023-04-27|
|   TXN_3160411|      2|          2.0|             4.0|              3|               1|      2023-06-11|
|   TXN_2602893|      7|          5.0|            20.0|              2|            NULL|      2023-03-31|
|   TXN_4433211|   NULL|          3.0|        